# Весь pipeline

Все работает, но не все закинуто в оболочку

## Видео в аудио

In [3]:
#нужные pip
%pip install ffmpeg
%pip install moviepy

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=5dbff42a573fef9042a7fff48e2709dc8bba0cde954ee0f3a31dac3c26029fea
  Stored in directory: /tmp/xdg_cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for urllib3<1.27,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 810.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 2.3 MB/s eta 0:00:00 0:00:01


In [ ]:
import subprocess
import os
import sys

def convert_video_to_audio_ffmpeg(video_file, output_ext="mp3"):
    """Converts video to audio directly using `ffmpeg` command
    with the help of subprocess module"""
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"], 
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)

In [ ]:
#случайное видио, просто проверить работу
#vf = sys.argv[1]
#convert_video_to_audio_ffmpeg('/content/IMG_0856.MOV')

## Аудио в текст

In [4]:
%pip install git+https://github.com/openai/whisper.git 

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tejvz6hc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tejvz6hc
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for urllib3<1.27,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 2

In [5]:
import whisper
import torch

In [6]:
model = whisper.load_model("medium")
#при надобности модель сохраняется
SAVE = False
if (SAVE):
    PATH = '/content/model.pkl'
    torch.save(model, PATH)
LOAD = False
# или подгружается
if (LOAD):
    model = torch.load(PATH)


100%|█████████████████████████████████████| 1.42G/1.42G [00:47<00:00, 32.5MiB/s]


In [7]:
model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-23): 24 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1024, out_features=1024, bias=True)
          (key): Linear(in_features=1024, out_features=1024, bias=False)
          (value): Linear(in_features=1024, out_features=1024, bias=True)
          (out): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (attn_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
        (mlp_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((

In [9]:
path_to_mp3 = '/home/jupyter/datasphere/project/data/audio6.mp3'
result = model.transcribe(path_to_mp3, language = 'ru')

In [10]:
#сохраняем транскрибированный текст
path_to_save_txt = "/home/jupyter/datasphere/project/res/medium/transcription6.txt"
with open(path_to_save_txt, "w") as output:
    output.write(str(result['text']))

In [12]:
import pandas as pd

In [13]:
#сохраняем csv с временными метками
path_to_save_csv = '/home/jupyter/datasphere/project/res/medium/res6.csv'
pd.DataFrame(result['segments']).to_csv(path_to_save_csv)

## Исправляем орфографию

In [15]:
%pip install transformers sentencepiece datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.8 MB/s eta 0:00:00a 0:00:010m
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/35/a8/36d8d7b3e46b377800d8dec47891cdf05842d1a2366909ae4a0c89fbc5e6/multiprocess-0.70.15-py310-none-any.whl.metadata


In [16]:
%pip install autocorrect

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 2.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=b5110ad4c69fb4bc4925b8f282d54ec2d02b66fc9f05ee9d5dc69e3d1766f650
  Stored in directory: /tmp/xdg_cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [17]:
path_txt_name = "/home/jupyter/datasphere/project/res/medium/transcription1.txt"

In [18]:
from autocorrect import Speller
spell = Speller('ru')

dictionary for this language not found, downloading...
__________________________________________________
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
done!


In [23]:
text_file = open(path_txt_name, "r")
data = text_file.read()
redacted = spell(data)
path_to_save_redacted_txt = "/home/jupyter/datasphere/project/res/medium/redacted1.txt"
file = open(path_to_save_redacted_txt, "w")
file.write(redacted)
file.close()

In [22]:
redacted

' Друзья, привет! С вами снова я Денис. Рад вас приветствовать на лекции. Сегодня мы будем говорить о массивах. На прошлой лекции мы познакомились с языком программирования C-sharp, с его основными характеристиками, а также решили блок базовой задачи. На семинарах вы практиковались в разработке циклических условных конструкций, применяли арифметические операторы, а также операции водовода. Сегодня мы вспомним основные понятия, связанные с массивами, их характеристики, поговорим о том, в каких задачах массивы могут применяться. Познакомимся, как осуществляется операция создания, заполнения, а также вывода массивов на экран и решим блок задач на их обработку. Ближе к концу лекции мы познакомимся с двумя разновидностями циклов FOR и FORGE, а также, друзья, поговорим о таком важном аспекте, как изучение английского для программистов. Друзья, перед тем как приступить к практике и начать решать задачи, давайте вспомним, а что из себя представляет массив и какими характеристиками он обладает.

## Получаем заголовок, структуру и вывод урока

In [26]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [33]:
path_txt_name = '/home/jupyter/datasphere/project/res/medium/transcription6.txt'
text_file = open(path_txt_name, "r")
article_text =  text_file.read()

In [34]:
#заголовок лекции
input_ids = tokenizer(
    [article_text],
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    max_length=4000,
    return_tensors="pt"
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=3,
    num_beams=5,
    early_stopping=True
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

На этом уроке мы рассказали о том, как работать с интерпретатором в интерактивном режиме и как запустить исходный код с помощью интерпретатора, а также о возможностях для тестирования и обработки данных.


In [35]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [36]:
# структура лекции
# для более хорошей работы требуется дообучение на размеченных данных
n = 10
l_text = [ i for i in chunks(article_text, -1*len(article_text)// n // 1 * -1)]
all_summary = ''
for text in l_text:
    input_ids = tokenizer(
        [text],
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        max_length=5000,
        return_tensors="pt"
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=3,
        num_beams=5,
        early_stopping=True
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    all_summary = all_summary + ' ' + summary
    print(summary)

Язык программирования Python — один из самых простых и понятных синтаксисов относительно других языков программирования, а также популярный для разработчиков и энтузиастов. Это означает, что для него существует множество готовых библиотек, которые решают реальные задачи.
Если говорить о языке Python, то это уже не первый язык, который применяется во многих властях, а также многие другие области применения, например, веб-разработка, машинное обучение и искусственный интеллект. Однако, как и любой код, код на этом языке можно делать быстро и удобно.
На этом уроке мы узнаем о интерпретаторе Python, о его интерактивном режиме и IDE по HRM, об IDE и получении данных со стандартного ввода.
В сегодняшнем обзоре курсов всех по языкам программирования мы расскажем о том, как запустить исходный код Python с помощью интерпретатора. Это абсолютно аналогично тому, что мы делали в интерактивной среде.
Интерактивность среда разработки IDE.PageArm может быть полезна для промышленной разработки, а такж

In [37]:
#вывод лекции
# для более хорошей работы требуется дообучение на размеченных данных, на данный момент модель не всегда понимает контекст
input_ids = tokenizer(
    [all_summary],
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    max_length=5000,
    return_tensors="pt"
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=3,
    num_beams=5,
    early_stopping=True
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

В сегодняшнем обзоре курсов всех по языкам программирования мы расскажем о том, как запустить исходный код на этом языке, а также о его интерактивном режиме и получении данных со стандартного ввода.


## Получение потенциальных определений

In [1]:
!spacy download ru_core_news_md

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 23.5 MB/s eta 0:00:00
  Obtaining dependency information for urllib3<1.27,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 673.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 1.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


2023-11-24 19:25:35.054045: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 19:25:36.352340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 19:25:39.757659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (4.0.0)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normal

In [2]:
import nltk.data
nltk.download('punkt')
tokenizer_1 = nltk.data.load('tokenizers/punkt/russian.pickle')
fp = open("/home/jupyter/datasphere/project/res/medium/transcription6.txt")

data = fp.read()
input_sequences = tokenizer_1.tokenize(data)
len(input_sequences)

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


168

In [3]:
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import spacy

2023-11-24 19:26:15.715711: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 19:26:15.780403: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-24 19:26:16.966986: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
nlp = spacy.load("ru_core_news_md")
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [6]:
def lemmatize(doc):
    words = []
    for token in doc:
        if (token.is_stop != True) and (token.is_punct != True) and\
            (token.is_space != True) and (token.is_digit != True):
            words.append(token.lemma_)
    return ' '.join(words)

In [7]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [8]:
import re
def find_noun(txt):
    txt = re.sub(r'[«.,"\'-?–:—!;–»]', '', txt)
    txt_n_1 = preprocess_text(txt)
    #txt_n_2 = lemmatize(nlp(txt_n_1))
    doc_n = nlp(txt_n_1)
    index = 0
    nounIndices = []
    for token in doc_n:
        # print(token.text, token.pos_, token.dep_, token.head.text)
        #print(token)
        if token.pos_ == 'NOUN':
            nounIndices.append(index)
        index = index + 1
    
    txt_n_1_split = txt_n_1.split()
    #print(nounIndices)
    #print(txt_n_1_split)
    return [txt_n_1_split[i] for i in nounIndices]

In [10]:
res = set()
fp = open("/home/jupyter/datasphere/project/res/medium/transcription6.txt")
data = fp.read()
input_sequences = tokenizer_1.tokenize(data)
for i in input_sequences:
    res = res | set(find_noun(i))

In [11]:
import pandas as pd
df = pd.DataFrame(res, columns =['words'])

In [ ]:
#df.to_excel('words.xlsx')

In [13]:
res_context = []
ind_seq = []
seq = []
last_len = 0
#print(j)
fp = open("/home/jupyter/datasphere/project/res/medium/transcription6.txt")
data = fp.read()
input_sequences = tokenizer_1.tokenize(data)
#print(len(input_sequences))
for i, seq_ in enumerate(input_sequences):
    #print(i)
    l_ = find_noun(seq_)
    ind_seq += [i + last_len]*len(l_)
    res_context = res_context + l_
seq = seq + input_sequences + ['']
    #print(last_len)

In [14]:
strings = []
for word in df['words']:
    idx = res_context.index(word)
    idx_seq = ind_seq[idx]
    strings.append(seq[idx_seq] + ' ' + seq[idx_seq+1])

In [15]:
df['string'] = strings
df

,words,string
0,значение,Тут некоторые значения по умолчанию нам подска...
1,сущность,Следующая сущность – это переменная. Что такое...
2,программа,"Это программа, выполняющая обострочный анализ,..."
3,приложение,Большой набор библиотек и встроенных инструмен...
4,функция,Это делается с помощью функции print. Давайте ...
...,...,...
141,пример,"И как и во всех предыдущих примерах, для тести..."
142,многий,"Из-за этого в сети можно найти много гайтов, о..."
143,баз,Большой набор библиотек и встроенных инструмен...
144,курс,Давайте выведем стандартный для начала курсов ...


In [16]:
df.to_csv('data_ruBERT_6.csv')

## Уточнение определений


In [17]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for urllib3<1.27,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.10.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

In [18]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset

In [19]:
df = pd.read_csv("data_ruBERT_6.csv")

In [20]:
df['w_s'] = df['words'] + ' ' + df['string']

In [21]:
def f(s):
    return ' '.join(s.split()[:512])

df['w_s'] = df['w_s'].apply( f )

In [29]:
df['label'] = 0

In [30]:
pred_text = df.reset_index()['w_s'].astype('str')
pred_label = df.reset_index()['label']

In [25]:
model_name = "/home/jupyter/datasphere/project/fine-tune-bert"

In [26]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [27]:
tokens_pred = tokenizer.batch_encode_plus(
    pred_text.values,
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

In [ ]:
%pip install accelerate -U

In [ ]:
%pip install transformers[torch]

In [31]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)

In [33]:
pred_dataset = Data(tokens_pred, pred_label)

In [38]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args)

In [40]:
def get_prediction():
    pred = trainer.predict(pred_dataset)
    labels = np.argmax(pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [41]:
pred

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [42]:
df['labels'] = pred

In [45]:
df_real  = df[df['labels'] == 1][['words','string']]
df_real

,words,string
1,сущность,Следующая сущность – это переменная. Что такое...
22,константа,"Литерал – это то же самое, что константа. Напр..."
41,литерал,"Начнем с литералы. Литерал – это то же самое, ..."
64,число,"Например, 123 – число. Это константа."
72,имя,Задается сначала имя переменной. Пусть будет var.
80,портативность,"И в-пятых, портативность. Код на Python, напис..."
84,образ,В языке Python переменная задается следующим о...
100,использование,"Во-первых, это простой для понимания и использ..."
119,синтаксис,"У языка Python, пожалуй, один из самых простых..."


In [46]:
df_real.to_csv('term6.csv')

## Создание определений

In [47]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [48]:
term = pd.read_csv('/home/jupyter/datasphere/project/term6.csv')

In [58]:
# для хорошего результата требуется дообучение
w = []
def_ = []
for index, row in term.iterrows():
    text =  row['words'] + " это - " + row['string']
    input_ids = tokenizer(
        [text],
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        max_length=1000,
        return_tensors="pt"
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=3,
        num_beams=5,
        early_stopping=True
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    w.append(row['words'])
    def_.append(summary)
    print(summary)

Следующая сущность – это переменная, а переменная – переменная. Что такое переменная в переменной, когда переменная является переменной.
Это то же самое, что константа, константа это - Литерал – число, которое является числом, а константа — число. Это - число числа, которые являются числами и числами. Если константа не будет, то число может быть меньше, чем константа.
Начнем с литералы. Литерал – это константа, константа – константа и константа.
Например, 123 – число. Это константа, константа – константа. Если число это число, то константа это константа и константа числа.
Имя переменной - Задается сначала имя это - Пусть будет var. Это переменная, а потом переменная переменной.
Python, написанный на одной платформе, спокойно запустится на другой, например, на Windows. Код на Python - и в-пятых, портативность это.
В языке Python переменная задается следующим образом. Задается имя переменной, а затем имя ее переменной.
Это простой для понимания и использования язык. Это очень простой, по

In [62]:
term_and_definition = pd.DataFrame({'term':w, 'definition':def_})

In [63]:
term_and_definition

,term,definition
0,сущность,"Следующая сущность – это переменная, а перемен..."
1,константа,"Это то же самое, что константа, константа это ..."
2,литерал,"Начнем с литералы. Литерал – это константа, ко..."
3,число,"Например, 123 – число. Это константа, констант..."
4,имя,Имя переменной - Задается сначала имя это - Пу...
5,портативность,"Python, написанный на одной платформе, спокойн..."
6,образ,В языке Python переменная задается следующим о...
7,использование,Это простой для понимания и использования язык...
8,синтаксис,Синтаксис это - язык интерпретируемый синтакси...


## Поиск временной метки

In [64]:
timecode = pd.read_csv('/home/jupyter/datasphere/project/res/medium/res6.csv')

In [65]:
timecode

,Unnamed: 0,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0,0.00,14.88,"Привет, меня зовут Владислав Эфимов, я работа...","[50364, 38932, 11, 6885, 46376, 41022, 3323, 2...",0.0,-0.197698,1.781457,0.036399
1,1,1,0,14.88,15.88,VK Рекламы.,"[51108, 691, 42, 6325, 2872, 35768, 698, 13, 5...",0.0,-0.197698,1.781457,0.036399
2,2,2,0,15.88,18.56,"Прежде чем приступить к изучению языка, я кратко","[51158, 8567, 48013, 12056, 5082, 14474, 3258,...",0.0,-0.197698,1.781457,0.036399
3,3,3,0,18.56,21.36,расскажу о преимуществах этого языка программ...,"[51292, 17399, 29294, 1000, 43228, 2165, 585, ...",0.0,-0.197698,1.781457,0.036399
4,4,4,0,21.36,24.44,какие задачи можно с помощью него решать и в ...,"[51432, 19971, 38793, 435, 8885, 776, 36387, 1...",0.0,-0.197698,1.781457,0.036399
...,...,...,...,...,...,...,...,...,...,...,...
263,263,263,81184,815.16,816.16,код в ней.,"[50530, 981, 1435, 740, 23227, 13, 50580]",0.0,-0.157119,1.854478,0.443487
264,264,264,81184,816.16,819.44,"Написали свою первую программу Hello World, а...","[50580, 28167, 3323, 4071, 23069, 11922, 3924,...",0.0,-0.157119,1.854478,0.443487
265,265,265,81184,819.44,822.98,"с такими понятиями как литерал, переменная, н...","[50744, 776, 2936, 11426, 7903, 21689, 31215, ...",0.0,-0.157119,1.854478,0.443487
266,266,266,81184,822.98,826.20,"выводить данные в консоль, а также считывать ...","[50921, 2840, 17533, 3258, 19582, 4970, 740, 6...",0.0,-0.157119,1.854478,0.443487


In [72]:
def find_term(term, timecode):
    for index, row in timecode.iterrows():
        if (term in row['text'].split()):
            print(int(row['start'] // 60), 'мин', int(row['start'] % 60), 'сек')
        

In [73]:
find_term('Интерпретатор', timecode)

9 мин 22 сек
9 мин 54 сек
